# Speciation Solution SymPy Code Generation
### Gayson's gas speciation example

In [ ]:
import pandas as pd
import numpy as np
import sympy as sym
import scipy.optimize as opt
from os import path
import sys
import fileinput
from thermoengine import coder, core, phases, model, equilibrate
sym.init_printing()

# Speciation Solution Properties
Assumptions:
- There are $nc$ components in the system
- There are $nc+ns$ species in the system.
 - The species are divided into $nb$ basis species ($nb$ == $nc$) and $ns$ non-basis (dependent) species
- The configurational entropy is described as a simple $x_i log(x_i)$ sum over all species
- The example below assumes ideal mixing amongst species

# Make gas properties (Grayson's code)

In [ ]:
t = 1796 - 300.0  # K
p = 1e-5  # bars
module_type = 'calib'
basis_list    = ['H1', 'He1', 'O1', 'Al1']
nonbasis_list = ['Al1H1', 'Al1H1O2', 'Al1O1', 'Al2', 'H2', 'H2O1', 'O2']
gas_list = basis_list + nonbasis_list

## Number of solution components, basis species and non-basis species

In [ ]:
nc = len(basis_list)
nb = nc
ns = len(nonbasis_list)

In [ ]:
coef_dict = {
    "H1" : {'k0': 20.786000000001557, 'k1': -1.492388094880245e-14, 'k2': 6.117226234539431e-17, 'k3': -1.4282994509661392e-19, 'k4': 2.139609514682541e-22, 'k5': -2.1938469964711776e-25, 'k6': 1.6014350411063657e-28, 'k7': -8.527147031356555e-32, 'k8': 3.357783787348501e-35, 'k9': -9.826073605388629e-39, 'k10': 2.1285971728361284e-42, 'k11': -3.364834905637717e-46, 'k12': 3.769709070697414e-50, 'k13': -2.8339551946289157e-54, 'k14': 1.281637284272436e-58, 'k15': -2.634511055906802e-63, 'T_r': 298.15, 'P_r': 1, 'S_TrPr': 114.71600000000001, 'H_TrPr': 217999.00000000003, 'R': 8.31446261815324},
    "He1" : {'k0': 20.786000000001557, 'k1': -1.492388094880245e-14, 'k2': 6.117226234539431e-17, 'k3': -1.4282994509661392e-19, 'k4': 2.139609514682541e-22, 'k5': -2.1938469964711776e-25, 'k6': 1.6014350411063657e-28, 'k7': -8.527147031356555e-32, 'k8': 3.357783787348501e-35, 'k9': -9.826073605388629e-39, 'k10': 2.1285971728361284e-42, 'k11': -3.364834905637717e-46, 'k12': 3.769709070697414e-50, 'k13': -2.8339551946289157e-54, 'k14': 1.281637284272436e-58, 'k15': -2.634511055906802e-63, 'T_r': 298.15, 'P_r': 1, 'S_TrPr': 126.152, 'H_TrPr': 0.0, 'R': 8.31446261815324},
    "O1" : {'k0': 27.096583549515778, 'k1': -0.03867110928500913, 'k2': 0.00011903845773799972, 'k3': -2.2820814929034287e-07, 'k4': 2.9641041168223947e-10, 'k5': -2.730744519090737e-13, 'k6': 1.8352849243888721e-16, 'k7': -9.155266591208966e-20, 'k8': 3.420892581659791e-23, 'k9': -9.590599589260897e-27, 'k10': 2.004970719536342e-30, 'k11': -3.075930796254847e-34, 'k12': 3.3592081170339484e-38, 'k13': -2.4703563114369085e-42, 'k14': 1.0959500426838366e-46, 'k15': -2.2150155559852535e-51, 'T_r': 298.15, 'P_r': 1, 'S_TrPr': 161.058, 'H_TrPr': 249173.0, 'R': 8.31446261815324},
    "Al1" : {'k0': 25.839382846834575, 'k1': -0.03638153846766955, 'k2': 0.0001253581361693125, 'k3': -2.619231142576971e-07, 'k4': 3.649544085438772e-10, 'k5': -3.5715550204682277e-13, 'k6': 2.533646045190727e-16, 'k7': -1.328335859168807e-19, 'k8': 5.200520335101602e-23, 'k9': -1.5242480649064922e-26, 'k10': 3.3257369674811575e-30, 'k11': -5.317966143392161e-34, 'k12': 6.046554339546409e-38, 'k13': -4.6248839534273176e-42, 'k14': 2.132071145555064e-46, 'k15': -4.473780164852519e-51, 'T_r': 298.15, 'P_r': 1, 'S_TrPr': 164.553, 'H_TrPr': 329699.0, 'R': 8.31446261815324},
    "Al1H1" : {'k0': 35.886411603313434, 'k1': -0.06790786155622007, 'k2': 0.0002512676676615485, 'k3': -4.399712922290216e-07, 'k4': 4.775683044321137e-10, 'k5': -3.517438714681003e-13, 'k6': 1.8300577637207767e-16, 'k7': -6.830388822920836e-20, 'k8': 1.8213085056856142e-23, 'k9': -3.3667330218586586e-27, 'k10': 3.9422622543174165e-31, 'k11': -1.980621217988875e-35, 'k12': -1.6673834215089603e-39, 'k13': 3.673519356326681e-43, 'k14': -2.580207744743585e-47, 'k15': 6.909118446299414e-52, 'T_r': 298.15, 'P_r': 1, 'S_TrPr': 187.864, 'H_TrPr': 259408.00000000003, 'R': 8.31446261815324},
    "Al1H1O2" : {'k0': 18.29973890543019, 'k1': 0.13762050611468235, 'k2': -5.1625523478916466e-05, 'k3': -3.652836879708368e-07, 'k4': 9.129890800052609e-10, 'k5': -1.1446034401241365e-12, 'k6': 9.249240058434729e-16, 'k7': -5.214547647330738e-19, 'k8': 2.1243558704942836e-22, 'k9': -6.347728697469884e-26, 'k10': 1.3930956386150302e-29, 'k11': -2.220097660992042e-33, 'k12': 2.4996099275916977e-37, 'k13': -1.884603194324491e-41, 'k14': 8.536130117032775e-46, 'k15': -1.7557674649524206e-50, 'T_r': 298.15, 'P_r': 1, 'S_TrPr': 254.391, 'H_TrPr': -460240.0, 'R': 8.31446261815324},
    "Al1O1" : {'k0': 28.83341095959386, 'k1': -0.025144457006330372, 'k2': 0.0002249366243162778, 'k3': -5.904337328602553e-07, 'k4': 8.533075343549667e-10, 'k5': -7.860364648272872e-13, 'k6': 5.002828615645185e-16, 'k7': -2.306873090054754e-19, 'k8': 7.910049551425489e-23, 'k9': -2.0388578297018592e-26, 'k10': 3.943987911274017e-30, 'k11': -5.645258313891538e-34, 'k12': 5.80192588237588e-38, 'k13': -4.04838853499277e-42, 'k14': 1.7167690343498575e-46, 'k15': -3.338155120542335e-51, 'T_r': 298.15, 'P_r': 1, 'S_TrPr': 218.386, 'H_TrPr': 66944.0, 'R': 8.31446261815324},
    "Al2" : {'k0': 18.1901877080998, 'k1': 0.09067641718022552, 'k2': 6.7970278211436774e-06, 'k3': -5.497791209790642e-07, 'k4': 1.2894604239929195e-09, 'k5': -1.6035848171955748e-12, 'k6': 1.2942677608139118e-15, 'k7': -7.2959552366423e-19, 'k8': 2.9720131567076504e-22, 'k9': -8.878276533153338e-26, 'k10': 1.9476282134170504e-29, 'k11': -3.1020954120156716e-33, 'k12': 3.490375600220861e-37, 'k13': -2.6297140850991143e-41, 'k14': 1.1902001329054644e-45, 'k15': -2.446162146827261e-50, 'T_r': 298.15, 'P_r': 1, 'S_TrPr': 233.53099999999998, 'H_TrPr': 487018.0, 'R': 8.31446261815324},
    "H2" : {'k0': 19.660706248979125, 'k1': 0.0798582204366822, 'k2': -0.0002855377272450818, 'k3': 5.84092076114598e-07, 'k4': -7.772982559901289e-10, 'k5': 7.292770499597736e-13, 'k6': -5.00688020848215e-16, 'k7': 2.5594914539255607e-19, 'k8': -9.812874739161492e-23, 'k9': 2.8218820851503534e-26, 'k10': -6.043448519944335e-30, 'k11': 9.481775452622107e-34, 'k12': -1.0570154522427e-37, 'k13': 7.920417535092067e-42, 'k14': -3.574295496937053e-46, 'k15': 7.337090320909886e-51, 'T_r': 298.15, 'P_r': 1, 'S_TrPr': 130.68, 'H_TrPr': 0.0, 'R': 8.31446261815324},
    "H2O1" : {'k0': 40.605147736379045, 'k1': -0.08102746252639997, 'k2': 0.0003526155000709938, 'k3': -8.073957114678312e-07, 'k4': 1.2138018642897184e-09, 'k5': -1.2502889428377463e-12, 'k6': 9.114960066736434e-16, 'k7': -4.821610984305613e-19, 'k8': 1.879996193662418e-22, 'k9': -5.438972621568167e-26, 'k10': 1.1643463384885733e-29, 'k11': -1.8193892350696435e-33, 'k12': 2.0161682556792162e-37, 'k13': -1.5004634688836166e-41, 'k14': 6.72333580900927e-46, 'k15': -1.3704586789124804e-50, 'T_r': 298.15, 'P_r': 1, 'S_TrPr': 188.834, 'H_TrPr': -241826.0, 'R': 8.31446261815324},
    "O2" : {'k0': 36.37405972115716, 'k1': -0.07609038186802722, 'k2': 0.00029648552317652487, 'k3': -5.596135245357751e-07, 'k4': 6.646269062106304e-10, 'k5': -5.464757260423527e-13, 'k6': 3.2630487973835284e-16, 'k7': -1.451007966603811e-19, 'k8': 4.8633316023376647e-23, 'k9': -1.2317762809181238e-26, 'k10': 2.3431338341198915e-30, 'k11': -3.293432375734748e-34, 'k12': 3.3165538134397923e-38, 'k13': -2.262474630466755e-42, 'k14': 9.362474257117735e-47, 'k15': -1.7740749756592712e-51, 'T_r': 298.15, 'P_r': 1, 'S_TrPr': 205.14700000000002, 'H_TrPr': 0.0, 'R': 8.31446261815324}
}
enki_formula = {
    "H1" :      ('H(1)', 1.0),
    "He1" :     ('He(1)', 1.0),
    "O1" :      ('O(1)', 1.0),
    "Al1" :     ('Al(1)', 1.0),
    "Al1H1" :   ('Al(0.5)H(0.5)', 2.0),
    "Al1H1O2" : ('Al(0.25)H(0.25)O(0.5)', 4.0),
    "Al1O1" :   ('Al(0.5)O(0.5)', 2.0),
    "Al2" :     ('Al(1)', 2.0),
    "H2" :      ('H(1)', 2.0),
    "H2O1" :    ('H(0.666666666666)O(0.333333333333)', 3.0),
    "O2" :      ('O(1)', 2.0)
}

In [ ]:
for key in coef_dict.keys():
    print ('{0:>10.10s} {1:15.2f} {2:10.2f}'.format(key, coef_dict[key]['H_TrPr'], coef_dict[key]['S_TrPr']))

## Generate endmembers from coder StdStateModel class

In [ ]:
modelCD = coder.StdStateModel()

In [ ]:
T = modelCD.get_symbol_for_t()
P = modelCD.get_symbol_for_p()
Tr = modelCD.get_symbol_for_tr()
Pr = modelCD.get_symbol_for_pr()

In [ ]:
STrPr,HTrPr,R,Scale = sym.symbols('S_TrPr H_TrPr R Scale')
params = [('H_TrPr','J',HTrPr), ('S_TrPr','J/K',STrPr), ('R', 'J/K', R), ('Scale','',Scale)]

In [ ]:
ncoeffs=15
coeff = sym.symbols('k0')
params += [('k0','NA', coeff)]
CpPr = coeff
for i in range(1, ncoeffs+1):
    coeff = sym.symbols('k'+str(i))
    CpPr += coeff*T**i
    params += [('k'+str(i),'NA', coeff)]
GTP = HTrPr/Scale + sym.integrate(CpPr,(T,Tr,T))/Scale - T*(STrPr + sym.integrate(CpPr/T,(T,Tr,T)))/Scale

In [ ]:
GTP

In [ ]:
modelCD.add_expression_to_model(GTP, params)
modelCD.set_module_name("gas_species")

In [ ]:
model_working_dir = "working"
!mkdir -p {model_working_dir}
%cd {model_working_dir}

In [ ]:
for gas in gas_list:
    formula = enki_formula[gas][0]
    print ('Processing', formula, '...')
    param_dict = coef_dict[gas]
    param_dict['Scale'] = enki_formula[gas][1]
    modelCD.create_code_module(phase=gas, formula=formula, params=param_dict, module_type=module_type, silent=True)
    if module_type == "calib":
        str_append = "calib"
    else:
        str_append = "calc"
    if gas == gas_list[0]:
        %cp gas_species.pyx endmembers.pyx
        file_name = '"'+gas+'_gas_species_'+str_append+'.c"'
    elif gas == gas_list[-1]:
        %cat gas_species.pyx >> endmembers.pyx
        file_name += ', "'+gas+'_gas_species_'+str_append+'.c"'
        last_filename = "'"+gas+"_gas_species_"+str_append+".c'"
    else:
        %cat gas_species.pyx >> endmembers.pyx
        file_name += ', "'+gas+'_gas_species_'+str_append+'.c"'

In [ ]:
%cp endmembers.pyx gas_species.pyx
with open('gas_species.pyxbld', 'r') as f:
    fold = f.read()
    f.close()
    fnew = fold.replace(last_filename, file_name)
with open('gas_species.pyxbld', 'w') as f:
    f.write(fnew)
    f.close()

In [ ]:
import gas_species
%cd ..

In [ ]:
if module_type == 'calib':
    calib_bool = True
else:
    calib_bool = False
modelDB_dict = {}    
for gas in gas_list:
    modelDB_dict[gas] = [gas,'pure']
modelDB = model.Database(database="CoderModule", calib=calib_bool, phase_tuple=('gas_species', modelDB_dict))

In [ ]:
print (t, p)
for gas in gas_list:
    print(gas, modelDB.get_phase(gas).gibbs_energy(t, p))

# Create conversion matrces to map basis to non-basis species
In this case use elements as a component basis (nc).  This does not have to be the case.  It is just convenient to do so here.  

Note that the chemical potentials of non-basis species are scaled so that element abundances sum to unity.  

Below is a list of basis ($b$) mole fractions, non-basis mole fractions ($s$) and chemical potential scaling factors applied in constructing the solution model

- H, $\mu_{1}$ = $\mu_{H}^{o}$, $b_{1}$

- He, $\mu_{2}$ = $\mu_{He}^{o}$, $b_{2}$

- O, $\mu_{3}$ = $\mu_{O}^{o}$, $b_{3}$

- Al, $\mu_{4}$ = $\mu_{Al}^{o}$, $b_{4}$

- (AlH)/2, $\mu_{5}$ = $\mu_{{Al_{\frac{1}{2}}}{H_{\frac{1}{2}}}}^{o}$ = $\frac{\mu_{AlH}^{o}}{2}$, $s_{1}$

- (AlHO2)/4, $\mu_{6}$ = $\mu_{{Al_{\frac{1}{4}}}{H_{\frac{1}{4}}}{O_{\frac{1}{2}}}}^{o}$ = $\frac{\mu_{AlHO_{2}}^{o}}{4}$, $s_{2}$

- (AlO)/2, $\mu_{7}$ = $\mu_{{Al_{\frac{1}{2}}}{O_{\frac{1}{2}}}}^{o}$ = $\frac{\mu_{AlO}^{o}}{2}$, $s_{3}$

- (Al2)/2, $\mu_{8}$ = $\mu_{Al}^{o}$ = $\frac{\mu_{Al_2}^{o}}{2}$, $s_{4}$

- (H2)/2, $\mu_{9}$ = $\mu_{H}^{o}$ = $\frac{\mu_{H_2}^{o}}{2}$, $s_{5}$

- (H2O)/3, $\mu_{10}$ = $\mu_{{H_{2}}{O}}^{o}$ = $\frac{\mu_{H_{2}O}^{o}}{3}$, $s_{6}$

- (O2)/2, $\mu_{11}$ = $\mu_{O}^{o}$ = $\frac{\mu_{O_2}^{o}}{2}$, $s_{7}$

In [ ]:
phase_list = [modelDB.get_phase(gas) for gas in basis_list]+[modelDB.get_phase(gas) for gas in nonbasis_list]
C = np.array([gas.props['element_comp'][0] for gas in phase_list])
elm_sys_ind = np.where(np.sum(C,axis=0) > 0)[0]
elm_sys = [core.chem.PERIODIC_ORDER[i] for i in elm_sys_ind]
C = C[:,elm_sys_ind]
print (elm_sys)
print (C)

#### Create basis and non-basis sub-matrices of C in SymPy

Partician this matrix into *basis* and *non-basis* submatrices  

Produce a SymPy Matrix symbol (sym.Matrix) from a list of lists (.tolist()) and turn all floating point elements of the matrix into rational numbers (sym.nsimplify(..., rational=True))  

In [ ]:
Cb = sym.nsimplify(sym.Matrix(C[:len(basis_list),:].tolist()), rational=True)
Cs = sym.nsimplify(sym.Matrix(C[len(basis_list):,:].tolist()), rational=True)
Cb, Cs

Form a reaction matrix, mapping basis species (columns) to non-basis species (rows)

In [ ]:
R = Cs * (Cb**-1)
R

# Create a speciation solution model
- $n$ is a vector of mole numbers of each basis species 
- $n_T$ is the total number of moles in the solution
- $X$ is a vector of mole fractions of components in the system
- $T$ is temperature in $K$
- $P$ is pressure in $bars$
- $\mu$ in Joules

In [ ]:
model = coder.SpeciationSolnModel(nc=nc, ns=ns, Cb=sym.matrix2numpy(Cb, dtype=float), Cs=sym.matrix2numpy(Cs, dtype=float), R=R, debug_print=True)

### Variables:
- $n$ is a vector of moles of elements in the system; length $nc$
- $nT$ is the sum of all the entries in $n$
- $X$ is a vector of mole fractions of all the elements in the system; length $nc$
- $mu$ is a vector of stabdard state chemical potentials of all species in solution; length $nc$+$ns$
- $s$ is a vector of moles of non-basis species; length $ns$
- $T$, $P$ are temperature (K) and pressure (bars)
- b_list is a list of basis species mole fractions defined using SypPy variables

In [ ]:
s = model.s
n = model.n
T = model.get_symbol_for_t()
P = model.get_symbol_for_p()
mu = model.mu
nT = model.nT
X = n/nT
b = sym.Matrix(model.b_list)
s,n,X,T,P,mu,nT,b

### Gibbs free energy of solution

In [ ]:
Rg = model.Rgas
Ghat = (b.transpose()*mu[:nc,0])[0]
for i in range(0,nc):
    Ghat += Rg*T*b[i]*sym.log(b[i]*P)
Ghat += (s.transpose()*mu[nc:,0])[0]
for i in range(0,ns):
    Ghat += Rg*T*s[i]*sym.log(s[i]*P)
G = nT*Ghat
G

In [ ]:
b1,b2,b3,b4 = sym.symbols('b1 b2 b3 b4')
G.subs([(b[0],b1), (b[1],b2), (b[2],b3), (b[3],b4)])

### Define the conditions of homogeneous equilibrium

In [ ]:
ordering_l = []
ordering_val = []
for i in range(0,ns):
    ordering_l.append(G.diff(s[i]))
    ordering_val.append(0.0)

### Define bound constarints on ordering parameters
Note that SymPy cannot yet deal with bound constraint reduction when there are multiple parameters in a given equation.  Hence, bounds on the basis variables are not considered.  

The bound constraint defined below, though required by the subclass of SpeciationSolnModel, is not used by the speciation solver.

In [ ]:
inequality_l = []
for i in range(0,nc):
    #inequality_l.append(0 <= b[i])
    #inequality_l.append(b[i] <= 1)
    pass
symbol_l = []
for i in range(0,ns):
    inequality_l.append(0 <= s[i])
    inequality_l.append(s[i] <= 1)
    symbol_l.append(s[i])
out = sym.reduce_inequalities(inequalities=inequality_l, symbols=symbol_l)
out

In [ ]:
model.add_expression_to_model(G, [('Rgas', 'J/K-mol', Rg)], ordering_functions=(ordering_l,s,ordering_val,out))

## Set up the model ...

In [ ]:
model.module = "Speciated_Solution"

... assign a formula string for code generation  

In [ ]:
formula_string = ''
conversion_string_l = []
test_string_l = []
for i, x in enumerate(elm_sys):
    formula_string += x + '[' + x + ']'
    conversion_string_l.append('[' + str(i) + ']=[' + x + ']')
    test_string_l.append('[' + str(i) + '] > 0.0')
formula_string, conversion_string_l, test_string_l

... assign a conversion string to map element concentrations to moles of end members

In [ ]:
model.formula_string = formula_string
model.conversion_string = conversion_string_l
model.test_string = test_string_l

# Use code printers to construct "C" package code

In [ ]:
model_working_dir = "working"
!mkdir -p {model_working_dir}
%cd {model_working_dir}

## Choose model type and create model
model_type is "fast" or "calib"

In [ ]:
model_type = "calib"
paramValues = {"Rgas":8.3143, "T_r":298.15, "P_r":1.0}
file_name.replace('"', '').replace('_calib.c', '').replace(' ', '').split(',')

In [ ]:
model.create_code_module(phase="IdealGas", params=paramValues, 
                         endmembers=file_name.replace('"', '').replace('_calib.c', '').replace(' ', '').split(','), 
                         prefix="cy", module_type=model_type, silent=False, minimal_deriv_set=True, debug=1)

## Load the module

In [ ]:
import Speciated_Solution
%cd ..

## Test and time the generated functions for IdealGas (T in K, P in bars)

In [ ]:
t = 1796 - 300
p = 1e-5

In [ ]:
# Anders and Grevesse 1989
# (abundances normalized to Si=1e6)
solar_dict = {"H":2.79e10,  #1
              "He":2.72e9,  #2
              "C":1.01e7,   #6
              "N":3.13e6,   #7
              "O":2.38e7,   #8
              "F":843,      #9
              "Ne":3.44e6,  #10
              "Na":5.74e4,  #11
              "Mg":1.074e6, #12
              "Al":8.49e4,  #13
              "Si":1e6,     #14
              "P":1.04e4,   #15
              "S":5.15e5,   #16
              "Cl":5240,    #17
              "Ar":1.01e5,  #18
              "K":3770,     #19
              "Ca":6.11e4,  #20
              "Ti":2400,    #22
              "Cr":1.35e4,  #24
              "Mn":9550,    #25
              "Fe":9.00e5,  #26
              "Co":2250,    #27
              "Ni":4.93e4,  #28
             }
n = np.array([solar_dict[elm] for elm in elm_sys])
print(elm_sys)
print(n)

## Available in both "Fast" and "Calib" code versions 
Execute the "fast" or "calibration" code metadata retrieval functions:

In [ ]:
try:
    print(Speciated_Solution.cy_IdealGas_Speciated_Solution_identifier())
    print(Speciated_Solution.cy_IdealGas_Speciated_Solution_name())
    print(Speciated_Solution.cy_IdealGas_Speciated_Solution_formula(t,p,n))
except AttributeError:
    pass
try:
    print(Speciated_Solution.cy_IdealGas_Speciated_Solution_calib_identifier())
    print(Speciated_Solution.cy_IdealGas_Speciated_Solution_calib_name())
    print(Speciated_Solution.cy_IdealGas_Speciated_Solution_calib_formula(t,p,n))
except AttributeError:
    pass

Test intrinsic element conversion routine ...

In [ ]:
try:
    e = np.zeros(106)
    sum = np.sum(n)
    for index in range(0,nc):
        end = Speciated_Solution.cy_IdealGas_Speciated_Solution_endmember_elements(index)
        for i in range(0,106):
            e[i] += end[i]*n[index]/sum
    nConv = Speciated_Solution.cy_IdealGas_Speciated_Solution_conv_elm_to_moles(e)
    for i in range(0,nc):
        print ('X[{0:d}] input {1:13.6e}, calc {2:13.6e}, diff {3:13.6e}'.format(
        i, n[i]/sum, nConv[i], nConv[i]-n[i]/sum))
    if not Speciated_Solution.cy_IdealGas_Speciated_Solution_test_moles(nConv):
        print ('Output of intrinsic composition calculation fails tests for permissible values.')
except AttributeError:
    pass
try:
    e = np.zeros(106)
    sum = np.sum(n)
    for index in range(0,nc):
        end = Speciated_Solution.cy_IdealGas_Speciated_Solution_calib_endmember_elements(index)
        for i in range(0,106):
            e[i] += end[i]*n[index]/sum
    nConv = Speciated_Solution.cy_IdealGas_Speciated_Solution_calib_conv_elm_to_moles(e)
    for i in range(0,nc):
        print ('X[{0:d}] input {1:13.6e}, calc {2:13.6e}, diff {3:13.6e}'.format(
        i, n[i]/sum, nConv[i], nConv[i]-n[i]/sum))
    if not Speciated_Solution.cy_IdealGas_Speciated_Solution_calib_test_moles(nConv):
        print ('Output of intrinsic composition calculation fails tests for permissible values.')
except AttributeError:
    pass

Test various conversion routines ...

In [ ]:
try:
    print (Speciated_Solution.cy_IdealGas_Speciated_Solution_calib_conv_moles_to_tot_moles(n))
    print (Speciated_Solution.cy_IdealGas_Speciated_Solution_calib_conv_moles_to_mole_frac(n))
    e = Speciated_Solution.cy_IdealGas_Speciated_Solution_calib_conv_moles_to_elm(n)
    print (e)
    print (Speciated_Solution.cy_IdealGas_Speciated_Solution_calib_conv_elm_to_moles(e))
    print (Speciated_Solution.cy_IdealGas_Speciated_Solution_calib_conv_elm_to_tot_moles(e))
    print (Speciated_Solution.cy_IdealGas_Speciated_Solution_calib_conv_elm_to_tot_grams(e))
except AttributeError:
    pass
try:
    print (Speciated_Solution.cy_IdealGas_Speciated_Solution_conv_moles_to_tot_moles(n))
    print (Speciated_Solution.cy_IdealGas_Speciated_Solution_conv_moles_to_mole_frac(n))
    e = Speciated_Solution.cy_IdealGas_Speciated_Solution_conv_moles_to_elm(n)
    print (e)
    print (Speciated_Solution.cy_IdealGas_Speciated_Solution_conv_elm_to_moles(e))
    print (Speciated_Solution.cy_IdealGas_Speciated_Solution_conv_elm_to_tot_moles(e))
    print (Speciated_Solution.cy_IdealGas_Speciated_Solution_conv_elm_to_tot_grams(e))
except AttributeError:
    pass

### Execute the standard thermodynamic property retrieval functions:

In [ ]:
Speciated_Solution.cy_IdealGas_Speciated_Solution_calib_g(t,p,n)

In [ ]:
fmt = "{0:<10.10s} {1:13.6e} {2:<10.10s}"
try:
    print(fmt.format('G', Speciated_Solution.cy_IdealGas_Speciated_Solution_g(t,p,n), 'J'))
    print(fmt.format('dGdT', Speciated_Solution.cy_IdealGas_Speciated_Solution_dgdt(t,p,n), 'J/K'))
    print(fmt.format('dGdP', Speciated_Solution.cy_IdealGas_Speciated_Solution_dgdp(t,p,n), 'J/bar'))
    print(fmt.format('d2GdT2', Speciated_Solution.cy_IdealGas_Speciated_Solution_d2gdt2(t,p,n), 'J/K^2'))
    print(fmt.format('d2GdTdP', Speciated_Solution.cy_IdealGas_Speciated_Solution_d2gdtdp(t,p,n), 'J/K-bar'))
    print(fmt.format('d2GdP2', Speciated_Solution.cy_IdealGas_Speciated_Solution_d2gdp2(t,p,n), 'J/bar^2'))
    print(fmt.format('d3GdT3', Speciated_Solution.cy_IdealGas_Speciated_Solution_d3gdt3(t,p,n), 'J/K^3'))
    print(fmt.format('d3GdT2dP', Speciated_Solution.cy_IdealGas_Speciated_Solution_d3gdt2dp(t,p,n), 'J/K^2-bar'))
    print(fmt.format('d3GdTdP2', Speciated_Solution.cy_IdealGas_Speciated_Solution_d3gdtdp2(t,p,n), 'J/K-bar^2'))
    print(fmt.format('d3GdP3', Speciated_Solution.cy_IdealGas_Speciated_Solution_d3gdp3(t,p,n), 'J/bar^3'))
    print(fmt.format('S', Speciated_Solution.cy_IdealGas_Speciated_Solution_s(t,p,n), 'J/K'))
    print(fmt.format('V', Speciated_Solution.cy_IdealGas_Speciated_Solution_v(t,p,n), 'J/bar'))
    print(fmt.format('Cv', Speciated_Solution.cy_IdealGas_Speciated_Solution_cv(t,p,n), 'J/K'))
    print(fmt.format('Cp', Speciated_Solution.cy_IdealGas_Speciated_Solution_cp(t,p,n), 'J/K'))
    print(fmt.format('dCpdT', Speciated_Solution.cy_IdealGas_Speciated_Solution_dcpdt(t,p,n), 'J/K^2'))
    print(fmt.format('alpha', Speciated_Solution.cy_IdealGas_Speciated_Solution_alpha(t,p,n), '1/K'))
    print(fmt.format('beta', Speciated_Solution.cy_IdealGas_Speciated_Solution_beta(t,p,n), '1/bar'))
    print(fmt.format('K', Speciated_Solution.cy_IdealGas_Speciated_Solution_K(t,p,n), 'bar'))
    print(fmt.format('Kp', Speciated_Solution.cy_IdealGas_Speciated_Solution_Kp(t,p,n), ''))
except AttributeError:
    pass
try:
    print(fmt.format('G', Speciated_Solution.cy_IdealGas_Speciated_Solution_calib_g(t,p,n), 'J'))
    print(fmt.format('dGdT', Speciated_Solution.cy_IdealGas_Speciated_Solution_calib_dgdt(t,p,n), 'J/K'))
    print(fmt.format('dGdP', Speciated_Solution.cy_IdealGas_Speciated_Solution_calib_dgdp(t,p,n), 'J/bar'))
    print(fmt.format('d2GdT2', Speciated_Solution.cy_IdealGas_Speciated_Solution_calib_d2gdt2(t,p,n), 'J/K^2'))
    print(fmt.format('d2GdTdP', Speciated_Solution.cy_IdealGas_Speciated_Solution_calib_d2gdtdp(t,p,n), 'J/K-bar'))
    print(fmt.format('d2GdP2', Speciated_Solution.cy_IdealGas_Speciated_Solution_calib_d2gdp2(t,p,n), 'J/bar^2'))
    print(fmt.format('d3GdT3', Speciated_Solution.cy_IdealGas_Speciated_Solution_calib_d3gdt3(t,p,n), 'J/K^3'))
    print(fmt.format('d3GdT2dP', Speciated_Solution.cy_IdealGas_Speciated_Solution_calib_d3gdt2dp(t,p,n), 'J/K^2-bar'))
    print(fmt.format('d3GdTdP2', Speciated_Solution.cy_IdealGas_Speciated_Solution_calib_d3gdtdp2(t,p,n), 'J/K-bar^2'))
    print(fmt.format('d3GdP3', Speciated_Solution.cy_IdealGas_Speciated_Solution_calib_d3gdp3(t,p,n), 'J/bar^3'))
    print(fmt.format('S', Speciated_Solution.cy_IdealGas_Speciated_Solution_calib_s(t,p,n), 'J/K'))
    print(fmt.format('V', Speciated_Solution.cy_IdealGas_Speciated_Solution_calib_v(t,p,n), 'J/bar'))
    print(fmt.format('Cv', Speciated_Solution.cy_IdealGas_Speciated_Solution_calib_cv(t,p,n), 'J/K'))
    print(fmt.format('Cp', Speciated_Solution.cy_IdealGas_Speciated_Solution_calib_cp(t,p,n), 'J/K'))
    print(fmt.format('dCpdT', Speciated_Solution.cy_IdealGas_Speciated_Solution_calib_dcpdt(t,p,n), 'J/K^2'))
    print(fmt.format('alpha', Speciated_Solution.cy_IdealGas_Speciated_Solution_calib_alpha(t,p,n), '1/K'))
    print(fmt.format('beta', Speciated_Solution.cy_IdealGas_Speciated_Solution_calib_beta(t,p,n), '1/bar'))
    print(fmt.format('K', Speciated_Solution.cy_IdealGas_Speciated_Solution_calib_K(t,p,n), 'bar'))
    print(fmt.format('Kp', Speciated_Solution.cy_IdealGas_Speciated_Solution_calib_Kp(t,p,n), ''))
except AttributeError:
    pass

### Execute functions that access endmember properties:

In [ ]:
fmt = "{0:<10.10s} {1:13.6e} {2:<15.15s}"
try:
    print ("number of components", Speciated_Solution.cy_IdealGas_Speciated_Solution_endmember_number())
    for index in range(0, nc):
        print ("{0:<20.20s}".format(Speciated_Solution.cy_IdealGas_Speciated_Solution_endmember_name(index)), end=' ')
        print ("{0:<20.20s}".format(Speciated_Solution.cy_IdealGas_Speciated_Solution_endmember_formula(index)))
        print ("mw: {0:10.2f}".format(Speciated_Solution.cy_IdealGas_Speciated_Solution_endmember_mw(index)))
        print (fmt.format('mu0', Speciated_Solution.cy_IdealGas_Speciated_Solution_endmember_mu0(index,t,p), 'J/mol'))
        print (fmt.format('dmu0dT', Speciated_Solution.cy_IdealGas_Speciated_Solution_endmember_dmu0dT(index,t,p), 'J/K-mol'))
        print (fmt.format('dmu0dP', Speciated_Solution.cy_IdealGas_Speciated_Solution_endmember_dmu0dP(index,t,p), 'J/bar-mol'))
        print (fmt.format('d2mu0dT2', Speciated_Solution.cy_IdealGas_Speciated_Solution_endmember_d2mu0dT2(index,t,p), 'J/K^2-mol'))
        print (fmt.format('d2mu0dTdP', Speciated_Solution.cy_IdealGas_Speciated_Solution_endmember_d2mu0dTdP(index,t,p), 'J/K-bar-mol'))
        print (fmt.format('d2mu0dP2', Speciated_Solution.cy_IdealGas_Speciated_Solution_endmember_d2mu0dP2(index,t,p), 'J/bar^2-mol'))
        print (fmt.format('d3mu0dT3', Speciated_Solution.cy_IdealGas_Speciated_Solution_endmember_d3mu0dT3(index,t,p), 'J/K^3-mol'))
        print (fmt.format('d3mu0dT2dP', Speciated_Solution.cy_IdealGas_Speciated_Solution_endmember_d3mu0dT2dP(index,t,p), 'J/K^2-bar-mol'))
        print (fmt.format('d3mu0dTdP2', Speciated_Solution.cy_IdealGas_Speciated_Solution_endmember_d3mu0dTdP2(index,t,p), 'J/K-bar^2-mol'))
        print (fmt.format('d3mu0dP3', Speciated_Solution.cy_IdealGas_Speciated_Solution_endmember_d3mu0dP3(index,t,p), 'J/bar^3-mol'))
        print ("Element array:")
        print (Speciated_Solution.cy_IdealGas_Speciated_Solution_endmember_elements(index))
        print ()
except AttributeError:
    pass
try:
    print ("number of components", Speciated_Solution.cy_IdealGas_Speciated_Solution_calib_endmember_number())
    for index in range(0, nc):
        print ("{0:<20.20s}".format(Speciated_Solution.cy_IdealGas_Speciated_Solution_calib_endmember_name(index)), end=' ')
        print ("{0:<20.20s}".format(Speciated_Solution.cy_IdealGas_Speciated_Solution_calib_endmember_formula(index)), end=' ')
        print ("mw: {0:10.2f}".format(Speciated_Solution.cy_IdealGas_Speciated_Solution_calib_endmember_mw(index)))
        print (fmt.format('mu0', Speciated_Solution.cy_IdealGas_Speciated_Solution_calib_endmember_mu0(index,t,p), 'J/mol'))
        print (fmt.format('dmu0dT', Speciated_Solution.cy_IdealGas_Speciated_Solution_calib_endmember_dmu0dT(index,t,p), 'J/K-mol'))
        print (fmt.format('dmu0dP', Speciated_Solution.cy_IdealGas_Speciated_Solution_calib_endmember_dmu0dP(index,t,p), 'J/bar-mol'))
        print (fmt.format('d2mu0dT2', Speciated_Solution.cy_IdealGas_Speciated_Solution_calib_endmember_d2mu0dT2(index,t,p), 'J/K^2-mol'))
        print (fmt.format('d2mu0dTdP', Speciated_Solution.cy_IdealGas_Speciated_Solution_calib_endmember_d2mu0dTdP(index,t,p), 'J/K-bar-mol'))
        print (fmt.format('d2mu0dP2', Speciated_Solution.cy_IdealGas_Speciated_Solution_calib_endmember_d2mu0dP2(index,t,p), 'J/bar^2-mol'))
        print (fmt.format('d3mu0dT3', Speciated_Solution.cy_IdealGas_Speciated_Solution_calib_endmember_d3mu0dT3(index,t,p), 'J/K^3-mol'))
        print (fmt.format('d3mu0dT2dP', Speciated_Solution.cy_IdealGas_Speciated_Solution_calib_endmember_d3mu0dT2dP(index,t,p), 'J/K^2-bar-mol'))
        print (fmt.format('d3mu0dTdP2', Speciated_Solution.cy_IdealGas_Speciated_Solution_calib_endmember_d3mu0dTdP2(index,t,p), 'J/K-bar^2-mol'))
        print (fmt.format('d3mu0dP3', Speciated_Solution.cy_IdealGas_Speciated_Solution_calib_endmember_d3mu0dP3(index,t,p), 'J/bar^3-mol'))
        print ("Element array:")
        print (Speciated_Solution.cy_IdealGas_Speciated_Solution_calib_endmember_elements(index))
        print ()
except AttributeError:
    pass

### Execute functions that access species properties:

In [ ]:
fmt = "{0:<10.10s} {1:13.6e} {2:<15.15s}"
try:
    print ("number of species", Speciated_Solution.cy_IdealGas_Speciated_Solution_species_number())
    for index in range(0, nc):
        print ("{0:<20.20s}".format(Speciated_Solution.cy_IdealGas_Speciated_Solution_species_name(index)), end=' ')
        print ("{0:<20.20s}".format(Speciated_Solution.cy_IdealGas_Speciated_Solution_species_formula(index)))
        print ("mw: {0:10.2f}".format(Speciated_Solution.cy_IdealGas_Speciated_Solution_species_mw(index)))
        print ("Element array:")
        print (Speciated_Solution.cy_IdealGas_Speciated_Solution_species_elements(index))
        print ()
except AttributeError:
    pass
try:
    print ("number of species", Speciated_Solution.cy_IdealGas_Speciated_Solution_calib_species_number())
    for index in range(0, nc):
        print ("{0:<20.20s}".format(Speciated_Solution.cy_IdealGas_Speciated_Solution_calib_species_name(index)), end=' ')
        print ("{0:<20.20s}".format(Speciated_Solution.cy_IdealGas_Speciated_Solution_calib_species_formula(index)), end=' ')
        print ("mw: {0:10.2f}".format(Speciated_Solution.cy_IdealGas_Speciated_Solution_calib_species_mw(index)))
        print ("Element array:")
        print (Speciated_Solution.cy_IdealGas_Speciated_Solution_calib_species_elements(index))
        print ()
except AttributeError:
    pass

### Execute functions for molar derivatives
#### First derivative vectors:

In [ ]:
def printResult(name, result, units):
    print ("{0:<10.10s}".format(name), end=' ')
    [print ("{0:13.6e}".format(x), end=' ') for x in result]
    print ("{0:<10.10s}".format(units))
def printLabels(n):
    print ("{0:<18.18s}".format(''), end=' ')
    [print ("[{0:3d}]{1:<8.8s}".format(idx, ''), end=' ') for idx in range(len(n))]
    print ()
printLabels(n)
try:
    printResult('dGdn', Speciated_Solution.cy_IdealGas_Speciated_Solution_dgdn(t,p,n), 'J/m')
    printResult('d2GdndT', Speciated_Solution.cy_IdealGas_Speciated_Solution_d2gdndt(t,p,n), 'J/K-m')
    printResult('d2GdndP', Speciated_Solution.cy_IdealGas_Speciated_Solution_d2gdndp(t,p,n), 'J/bar-m')
    printResult('d3GdndT2', Speciated_Solution.cy_IdealGas_Speciated_Solution_d3gdndt2(t,p,n), 'J/K^2-m')
    printResult('d3GdndTdP', Speciated_Solution.cy_IdealGas_Speciated_Solution_d3gdndtdp(t,p,n), 'J/K-bar-m')
    printResult('d3GdndP2', Speciated_Solution.cy_IdealGas_Speciated_Solution_d3gdndp2(t,p,n), 'J/bar^2-m')
    printResult('d4GdndT3', Speciated_Solution.cy_IdealGas_Speciated_Solution_d4gdndt3(t,p,n), 'J/K^3-m')
    printResult('d4GdndT2dP', Speciated_Solution.cy_IdealGas_Speciated_Solution_d4gdndt2dp(t,p,n), 'J/K^2-bar-m')
    printResult('d4GdndTdP2', Speciated_Solution.cy_IdealGas_Speciated_Solution_d4gdndtdp2(t,p,n), 'J/K-bar^2-m')
    printResult('d4GdndP3', Speciated_Solution.cy_IdealGas_Speciated_Solution_d4gdndp3(t,p,n), 'J/bar^3-m')
except AttributeError:
    pass
try:
    printResult('dGdn', Speciated_Solution.cy_IdealGas_Speciated_Solution_calib_dgdn(t,p,n), 'J/m')
    printResult('d2GdndT', Speciated_Solution.cy_IdealGas_Speciated_Solution_calib_d2gdndt(t,p,n), 'J/K-m')
    printResult('d2GdndP', Speciated_Solution.cy_IdealGas_Speciated_Solution_calib_d2gdndp(t,p,n), 'J/bar-m')
    printResult('d3GdndT2', Speciated_Solution.cy_IdealGas_Speciated_Solution_calib_d3gdndt2(t,p,n), 'J/K^2-m')
    printResult('d3GdndTdP', Speciated_Solution.cy_IdealGas_Speciated_Solution_calib_d3gdndtdp(t,p,n), 'J/K-bar-m')
    printResult('d3GdndP2', Speciated_Solution.cy_IdealGas_Speciated_Solution_calib_d3gdndp2(t,p,n), 'J/bar^2-m')
    printResult('d4GdndT3', Speciated_Solution.cy_IdealGas_Speciated_Solution_calib_d4gdndt3(t,p,n), 'J/K^3-m')
    printResult('d4GdndT2dP', Speciated_Solution.cy_IdealGas_Speciated_Solution_calib_d4gdndt2dp(t,p,n), 'J/K^2-bar-m')
    printResult('d4GdndTdP2', Speciated_Solution.cy_IdealGas_Speciated_Solution_calib_d4gdndtdp2(t,p,n), 'J/K-bar^2-m')
    printResult('d4GdndP3', Speciated_Solution.cy_IdealGas_Speciated_Solution_calib_d4gdndp3(t,p,n), 'J/bar^3-m')
except AttributeError:
    pass    

#### The Hessian matrix (molar second derivative matrix) is stored as a compact linear array
A function is provided to map matrix indices to compact storage 1-D array indices

In [ ]:
for i in range(1,nc+1):
    print ("[ ", end=' ')
    for j in range (1,nc+1):
        print ((i,j), end=' ')
    print (']     [', end=' ')
    for j in range (1,nc+1):
        print (model.symmetric_index_from_2d_array(elm=(i,j)), end=' ')
    print (']')

In [ ]:
def printResult(name, result, units):
    print ("{0:<10.10s}".format(name), end=' ')
    [print ("{0:13.6e}".format(x), end=' ') for x in result]
    print ("{0:<10.10s}".format(units))
def printLabels(n):
    print ("{0:<18.18s}".format(''), end=' ')
    maxIdx = int(len(n)*(len(n)-1)/2 + len(n))
    [print ("[{0:3d}]{1:<8.8s}".format(idx, ''), end=' ') for idx in range(maxIdx)]
    print ()
printLabels(n)
try:
    printResult('d2Gdn2', Speciated_Solution.cy_IdealGas_Speciated_Solution_d2gdn2(t,p,n), 'J/m^2')
    printResult('d3Gdn2dT', Speciated_Solution.cy_IdealGas_Speciated_Solution_d3gdn2dt(t,p,n), 'J/K-m^2')
    printResult('d3Gdn2dP', Speciated_Solution.cy_IdealGas_Speciated_Solution_d3gdn2dp(t,p,n), 'J/bar-m^2')
    printResult('d4Gdn2dT2', Speciated_Solution.cy_IdealGas_Speciated_Solution_d4gdn2dt2(t,p,n), 'J/K^2-m^2')
    printResult('d4Gdn2dTdP', Speciated_Solution.cy_IdealGas_Speciated_Solution_d4gdn2dtdp(t,p,n), 'J/K-bar-m^2')
    printResult('d4Gdn2dP2', Speciated_Solution.cy_IdealGas_Speciated_Solution_d4gdn2dp2(t,p,n), 'J/bar^2-m^2')
    printResult('d5Gdn2dT3', Speciated_Solution.cy_IdealGas_Speciated_Solution_d5gdn2dt3(t,p,n), 'J/K^3-m^2')
    printResult('d5Gdn2dT2dP', Speciated_Solution.cy_IdealGas_Speciated_Solution_d5gdn2dt2dp(t,p,n), 'J/K^2-bar-m^2')
    printResult('d5Gdn2dTdP2', Speciated_Solution.cy_IdealGas_Speciated_Solution_d5gdn2dtdp2(t,p,n), 'J/K-bar^2-m^2')
    printResult('d5Gdn2dP3', Speciated_Solution.cy_IdealGas_Speciated_Solution_d5gdn2dp3(t,p,n), 'J/bar^3-m^2')
except AttributeError:
    pass
try:
    printResult('d2Gdn2', Speciated_Solution.cy_IdealGas_Speciated_Solution_calib_d2gdn2(t,p,n), 'J/m^2')
    printResult('d3Gdn2dT', Speciated_Solution.cy_IdealGas_Speciated_Solution_calib_d3gdn2dt(t,p,n), 'J/K-m^2')
    printResult('d3Gdn2dP', Speciated_Solution.cy_IdealGas_Speciated_Solution_calib_d3gdn2dp(t,p,n), 'J/bar-m^2')
    printResult('d4Gdn2dT2', Speciated_Solution.cy_IdealGas_Speciated_Solution_calib_d4gdn2dt2(t,p,n), 'J/K^2-m^2')
    printResult('d4Gdn2dTdP', Speciated_Solution.cy_IdealGas_Speciated_Solution_calib_d4gdn2dtdp(t,p,n), 'J/K-bar-m^2')
    printResult('d4Gdn2dP2', Speciated_Solution.cy_IdealGas_Speciated_Solution_calib_d4gdn2dp2(t,p,n), 'J/bar^2-m^2')
    printResult('d5Gdn2dT3', Speciated_Solution.cy_IdealGas_Speciated_Solution_calib_d5gdn2dt3(t,p,n), 'J/K^3-m^2')
    printResult('d5Gdn2dT2dP', Speciated_Solution.cy_IdealGas_Speciated_Solution_calib_d5gdn2dt2dp(t,p,n), 'J/K^2-bar-m^2')
    printResult('d5Gdn2dTdP2', Speciated_Solution.cy_IdealGas_Speciated_Solution_calib_d5gdn2dtdp2(t,p,n), 'J/K-bar^2-m^2')
    printResult('d5Gdn2dP3', Speciated_Solution.cy_IdealGas_Speciated_Solution_calib_d5gdn2dp3(t,p,n), 'J/bar^3-m^2')
except AttributeError:
    pass

#### The 3-D Tensor (molar third derivative tensor) is stored as a compact linear array
A function is provided to map matrix indices to compact storage 1-D array indices:  
If $n_c$ represents the number of components in the solution, and  
if $n_d$ represents the dimensionality of molar derivative (in this case 3), then  
the number of numerically ordered permutations of $n_c$ molar derivatives taken $n_d$ at a time is:

In [ ]:
n_c,n_d = sym.symbols('n_c n_d')
q = sym.factorial(n_c+n_d-1)/sym.factorial(n_d)/sym.factorial(n_c-1)
q

Substituting $n_d$ equal to 3 and simplifying gives:

In [ ]:
q = sym.simplify(q.subs(n_d,3))
q

and, for the number of components in this solution, there will be the following number of unique terms in the third derivative tensor:

In [ ]:
q.subs(n_c,nc)

A function is provided to map matrix indices to compact storage 1-D array indices

In [ ]:
for i in range(1,nc+1):
    for j in range (1,nc+1):
        print ("[", end=' ')
        for k in range (1,nc+1):
            print ("{0:1d}{1:1d}{2:1d}".format(i,j,k), end=' ')
        print ('] ', end=' ')
    print ('  ->  ', end=' ')
    for j in range (1,nc+1):
        print ("[", end=' ')
        for k in range (1,nc+1):
            print (model.symmetric_index_from_3d_array(elm=(i,j,k)), end=' ')
        print ('] ', end=' ')
    print ('')

In [ ]:
def printResult(name, result, units):
    print ("{0:<10.10s}".format(name), end=' ')
    [print ("{0:10.3e}".format(x), end=' ') for x in result]
    print ("{0:<14.14s}".format(units))
def printLabels(n):
    print ("{0:<15.15s}".format(''), end=' ')
    maxIdx = int(len(n)*(len(n)+1)*(len(n)+2)/6)
    [print ("[{0:3d}]{1:<5.5s}".format(idx, ''), end=' ') for idx in range(maxIdx)]
    print ()
printLabels(n)
try:
    printResult('d3Gdn3', Speciated_Solution.cy_IdealGas_Speciated_Solution_d3gdn3(t,p,n), 'J/m^3')
    printResult('d4Gdn3dT', Speciated_Solution.cy_IdealGas_Speciated_Solution_d4gdn3dt(t,p,n), 'J/K-m^3')
    printResult('d4Gdn3dP', Speciated_Solution.cy_IdealGas_Speciated_Solution_d4gdn3dp(t,p,n), 'J/bar-m^3')
    printResult('d5Gdn3dT2', Speciated_Solution.cy_IdealGas_Speciated_Solution_d5gdn3dt2(t,p,n), 'J/K^2-m^3')
    printResult('d5Gdn3dTdP', Speciated_Solution.cy_IdealGas_Speciated_Solution_d5gdn3dtdp(t,p,n), 'J/K-bar-m^3')
    printResult('d5Gdn3dP2', Speciated_Solution.cy_IdealGas_Speciated_Solution_d5gdn3dp2(t,p,n), 'J/bar^2-m^3')
    printResult('d6Gdn3dT3', Speciated_Solution.cy_IdealGas_Speciated_Solution_d6gdn3dt3(t,p,n), 'J/K^3-m^3')
    printResult('d6Gdn3dT2dP', Speciated_Solution.cy_IdealGas_Speciated_Solution_d6gdn3dt2dp(t,p,n), 'J/K^2-bar-m^3')
    printResult('d6Gdn3dTdP2', Speciated_Solution.cy_IdealGas_Speciated_Solution_d6gdn3dtdp2(t,p,n), 'J/K-bar^2-m^3')
    printResult('d6Gdn3dP3', Speciated_Solution.cy_IdealGas_Speciated_Solution_d6gdn3dp3(t,p,n), 'J/bar^3-m^3')
except AttributeError:
    pass
try:
    printResult('d3Gdn3', Speciated_Solution.cy_IdealGas_Speciated_Solution_calib_d3gdn3(t,p,n), 'J/m^3')
    printResult('d4Gdn3dT', Speciated_Solution.cy_IdealGas_Speciated_Solution_calib_d4gdn3dt(t,p,n), 'J/K-m^3')
    printResult('d4Gdn3dP', Speciated_Solution.cy_IdealGas_Speciated_Solution_calib_d4gdn3dp(t,p,n), 'J/bar-m^3')
    printResult('d5Gdn3dT2', Speciated_Solution.cy_IdealGas_Speciated_Solution_calib_d5gdn3dt2(t,p,n), 'J/K^2-m^3')
    printResult('d5Gdn3dTdP', Speciated_Solution.cy_IdealGas_Speciated_Solution_calib_d5gdn3dtdp(t,p,n), 'J/K-bar-m^3')
    printResult('d5Gdn3dP2', Speciated_Solution.cy_IdealGas_Speciated_Solution_calib_d5gdn3dp2(t,p,n), 'J/bar^2-m^3')
    printResult('d6Gdn3dT3', Speciated_Solution.cy_IdealGas_Speciated_Solution_calib_d6gdn3dt3(t,p,n), 'J/K^3-m^3')
    printResult('d6Gdn3dT2dP', Speciated_Solution.cy_IdealGas_Speciated_Solution_calib_d6gdn3dt2dp(t,p,n), 'J/K^2-bar-m^3')
    printResult('d6Gdn3dTdP2', Speciated_Solution.cy_IdealGas_Speciated_Solution_calib_d6gdn3dtdp2(t,p,n), 'J/K-bar^2-m^3')
    printResult('d6Gdn3dP3', Speciated_Solution.cy_IdealGas_Speciated_Solution_calib_d6gdn3dp3(t,p,n), 'J/bar^3-m^3')
except AttributeError:
    pass

## Test and time the generated functions for IdealGas

Time the code

In [ ]:
try:
    %timeit Speciated_Solution.cy_IdealGas_Speciated_Solution_g(t, p, n)
except AttributeError:
    pass
try:
    %timeit Speciated_Solution.cy_IdealGas_Speciated_Solution_calib_g(t, p, n) 
except AttributeError:
    pass

## Methods available only in the "Calib" versions of generated code
### Execute the parameter value/metadata functions.  
These functions are only defined for the "calibration" model code implementation:

In [ ]:
nparam = 0

In [ ]:
try:
    nparam = Speciated_Solution.cy_IdealGas_Speciated_Solution_get_param_number()
    names = Speciated_Solution.cy_IdealGas_Speciated_Solution_get_param_names()
    units = Speciated_Solution.cy_IdealGas_Speciated_Solution_get_param_units()
    values = Speciated_Solution.cy_IdealGas_Speciated_Solution_get_param_values()
    fmt = "{0:<10.10s} {1:13.6e} {2:13.6e} {3:<10.10s}"
    for i in range(0,nparam):
        print(fmt.format(names[i], values[i], Speciated_Solution.cy_IdealGas_Speciated_Solution_get_param_value(i), units[i]))
except AttributeError:
    pass

### Functions that allow modification of the array of parameter values

In [ ]:
try:
    values[1] = 100.0
    Speciated_Solution.cy_IdealGas_Speciated_Solution_set_param_values(values)
    fmt = "{0:<10.10s} {1:13.6e} {2:13.6e} {3:<10.10s}"
    for i in range(0,nparam):
        print(fmt.format(names[i], values[i], Speciated_Solution.cy_IdealGas_Speciated_Solution_get_param_value(i), units[i]))
except (AttributeError, NameError):
    pass

### Functions that allow modification of a particular parameter value

In [ ]:
try:
    Speciated_Solution.cy_IdealGas_Speciated_Solution_set_param_value(1, 1.0)
    fmt = "{0:<10.10s} {1:13.6e} {2:13.6e} {3:<10.10s}"
    for i in range(0,nparam):
        print(fmt.format(names[i], values[i], Speciated_Solution.cy_IdealGas_Speciated_Solution_get_param_value(i), units[i]))
except AttributeError:
    pass

### Functions that evaluate parameter derivatives ...

In [ ]:
try:
    fmt = "    {0:<10.10s} {1:13.6e}"
    for i in range(0, nparam):
        print ('Derivative with respect to parameter: ', names[i], ' of')
        print (fmt.format('G', Speciated_Solution.cy_IdealGas_Speciated_Solution_dparam_g(t, p, n, i)))
        print (fmt.format('dGdT', Speciated_Solution.cy_IdealGas_Speciated_Solution_dparam_dgdt(t, p, n, i)))
        print (fmt.format('dGdP', Speciated_Solution.cy_IdealGas_Speciated_Solution_dparam_dgdp(t, p, n, i)))
        print (fmt.format('d2GdT2', Speciated_Solution.cy_IdealGas_Speciated_Solution_dparam_d2gdt2(t, p, n, i)))
        print (fmt.format('d2GdTdP', Speciated_Solution.cy_IdealGas_Speciated_Solution_dparam_d2gdtdp(t, p, n, i)))
        print (fmt.format('d2GdP2', Speciated_Solution.cy_IdealGas_Speciated_Solution_dparam_d2gdp2(t, p, n, i)))
        print (fmt.format('d3GdT3', Speciated_Solution.cy_IdealGas_Speciated_Solution_dparam_d3gdt3(t, p, n, i)))
        print (fmt.format('d3GdT2dP', Speciated_Solution.cy_IdealGas_Speciated_Solution_dparam_d3gdt2dp(t, p, n, i)))
        print (fmt.format('d3GdTdP2', Speciated_Solution.cy_IdealGas_Speciated_Solution_dparam_d3gdtdp2(t, p, n, i)))
        print (fmt.format('d3GdP3', Speciated_Solution.cy_IdealGas_Speciated_Solution_dparam_d3gdp3(t, p, n, i)))
except (AttributeError, TypeError):
    pass

### Parameter derivatives of the chemical potential

In [ ]:
def printResult(name, result, units):
    print ("dmu[*]/d {0:<10.10s}".format(name), end=' ')
    [print ("{0:13.6e}".format(x), end=' ') for x in result]
    print ("{0:<12.12s}".format(units))
def printLabels(n):
    print ("         {0:<18.18s}".format(''), end=' ')
    [print ("[{0:3d}]{1:<8.8s}".format(idx, ''), end=' ') for idx in range(len(n))]
    print ()
try:
    printLabels(n)
    for i in range(0, nparam):
        result = Speciated_Solution.cy_IdealGas_Speciated_Solution_dparam_dgdn(t,p,n, i)
        printResult(names[i], result, 'J/m^2/p-unit')
except AttributeError:
    pass    

### Execute the parameter value/metadata functions for endmembers.  
Use the Potassium IdealGas as an example.  Alternatively, other endmembers can be accessed at:
- Speciated_Solution.cy_High_Albite_berman_(method ...)
- Speciated_Solution.cy_Anorthite_berman_(method ...)
- Speciated_Solution.cy_Potassium_IdealGas_berman_(method ...)

In [ ]:
try:
    np = Speciated_Solution.cy_IdealGas_Speciated_Solution_get_param_number()
    names = Speciated_Solution.cy_IdealGas_Speciated_Solution_get_param_names()
    units = Speciated_Solution.cy_IdealGas_Speciated_Solution_get_param_units()
    values = Speciated_Solution.cy_IdealGas_Speciated_Solution_get_param_values()
    fmt = "{0:<10.10s} {1:13.6e} {2:13.6e} {3:<10.10s}"
    for i in range(0,np):
        print(fmt.format(names[i], values[i], Speciated_Solution.cy_IdealGas_Speciated_Solution_get_param_value(i), units[i]))
except AttributeError:
    pass

Test the functions that allow modification of the array of parameter values

In [ ]:
try:
    values[1] = 100.0
    Speciated_Solution.cy_IdealGas_Speciated_Solution_set_param_values(values)
    fmt = "{0:<10.10s} {1:13.6e} {2:13.6e} {3:<10.10s}"
    for i in range(0,np):
        print(fmt.format(names[i], values[i], Speciated_Solution.cy_IdealGas_Speciated_Solution_get_param_value(i), units[i]))
except (AttributeError, NameError):
    pass

Test the functions that allow modification of a particular parameter value

In [ ]:
try:
    Speciated_Solution.cy_IdealGas_Speciated_Solution_set_param_value(1, 1.0)
    fmt = "{0:<10.10s} {1:13.6e} {2:13.6e} {3:<10.10s}"
    for i in range(0,np):
        print(fmt.format(names[i], values[i], Speciated_Solution.cy_IdealGas_Speciated_Solution_get_param_value(i), units[i]))
except AttributeError:
    pass

Evaluate parameter derivatives ...

In [ ]:
try:
    fmt = "    {0:<10.10s} {1:13.6e}"
    for i in range(0, np):
        print ('Derivative with respect to parameter: ', names[i], ' of')
        print (fmt.format('G', Speciated_Solution.cy_IdealGas_Speciated_Solution_dparam_g(t, p, n, i)))
        print (fmt.format('dGdT', Speciated_Solution.cy_IdealGas_Speciated_Solution_dparam_dgdt(t, p, n, i)))
        print (fmt.format('dGdP', Speciated_Solution.cy_IdealGas_Speciated_Solution_dparam_dgdp(t, p, n, i)))
        print (fmt.format('d2GdT2', Speciated_Solution.cy_IdealGas_Speciated_Solution_dparam_d2gdt2(t, p, n, i)))
        print (fmt.format('d2GdTdP', Speciated_Solution.cy_IdealGas_Speciated_Solution_dparam_d2gdtdp(t, p, n, i)))
        print (fmt.format('d2GdP2', Speciated_Solution.cy_IdealGas_Speciated_Solution_dparam_d2gdp2(t, p, n, i)))
        print (fmt.format('d3GdT3', Speciated_Solution.cy_IdealGas_Speciated_Solution_dparam_d3gdt3(t, p, n, i)))
        print (fmt.format('d3GdT2dP', Speciated_Solution.cy_IdealGas_Speciated_Solution_dparam_d3gdt2dp(t, p, n, i)))
        print (fmt.format('d3GdTdP2', Speciated_Solution.cy_IdealGas_Speciated_Solution_dparam_d3gdtdp2(t, p, n, i)))
        print (fmt.format('d3GdP3', Speciated_Solution.cy_IdealGas_Speciated_Solution_dparam_d3gdp3(t, p, n, i)))
except (AttributeError, TypeError):
    pass

# Test an equilibrium calculation

In [ ]:
from thermoengine import model, equilibrate

In [ ]:
if module_type == 'calib':
    modelGas = model.Database(database="CoderModule", calib=True, 
                              phase_tuple=('Speciated_Solution', {'Gas':['IdealGas','solution']}))
else:
    modelGas = model.Database(database="CoderModule", calib=False, 
                              phase_tuple=('Speciated_Solution', {'Gas':['IdealGas','solution']}))
Gas = modelGas.get_phase('Gas')

In [ ]:
Crn = model.Database(liq_mod='v1.0').get_phase("Crn")
t, p, Crn.gibbs_energy(t, p)

### Try to precipitate corundum or liquid water from the gas

In [ ]:
phs_sys = [Gas, Crn]

In [ ]:
equil = equilibrate.Equilibrate(elm_sys, phs_sys)

In [ ]:
n = n/n.sum()
n

In [ ]:
state = equil.execute(t, p, bulk_comp=10.0*n, debug=3)
state.print_state()